In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
import numpy as np
import palettable
import pandas as pd 
import geopandas as gpd 

In [ ]:


# grab Bamako_7 from palettable.scientific.sequential
#cmap = palettable.scientific.sequential.Bamako_7.mpl_colormap
# use cividis 
cmap = matplotlib.cm.get_cmap('viridis')

def format_range(lower, upper):
    """Format a range of numbers with k/M suffixes"""
    def format_single(x):
        if x < 1000:
            return str(int(x))
        elif x < 1000000:
            return f'{int(x/1000)}k'
        else:
            return f'{int(x/1000000)}M'
    return f'{format_single(lower)}-{format_single(upper)}'

def create_intermediate_bins(start, end, base=10):
    """Create intermediate bins between powers of base"""
    start_exp = np.ceil(np.log10(start))
    end_exp = np.floor(np.log10(end))
    
    bins = [start]
    
    for exp in np.arange(start_exp, end_exp + 1):
        main_value = base ** exp
        bins.append(main_value)
        
        intermediate = 5 * main_value
        if intermediate <= end:
            bins.append(intermediate)
    
    if bins[-1] != end:
        bins.append(end)
    
    return np.array(bins)

# Control colorbar visibility
SHOW_COLORBAR = True

# Setup the figure
fig, ax = plt.subplots(figsize=(10,10))

# Get data range
data_min = cts_nyc['nframes'].min()
data_max = cts_nyc['nframes'].max()
smallest_nonzero = cts_nyc['nframes'].value_counts().sort_index().iloc[0:].head(1).index[0]

# Create bins and colormap
bins = create_intermediate_bins(smallest_nonzero, 50000)
#cmap = matplotlib.cm.get_cmap('viridis')
norm = BoundaryNorm(bins, cmap.N)

# Plot map
cts_nyc.boundary.plot(ax=ax, color='black', linewidth=0.5)
plot = cts_nyc.plot(column='nframes', ax=ax, legend=False, 
                     cmap=cmap, norm=norm, edgecolor='white', linewidth=0.1)
ax.axis('off')

# Create vertical colorbar in top left
if SHOW_COLORBAR:
    cax = fig.add_axes([0.2, 0.65, 0.02, 0.2])  # [left, bottom, width, height]
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')

# Calculate tick positions and set colorbar properties if shown
if SHOW_COLORBAR:
    tick_positions = [(bins[i] + bins[i+1])/2 for i in range(len(bins)-1)]
    cbar.set_ticks(tick_positions)

    # Create range labels
    tick_labels = [format_range(bins[i], bins[i+1]) for i in range(len(bins)-1)]
    cbar.ax.set_yticklabels(tick_labels, rotation=0)
    
    cbar.ax.yaxis.set_ticks_position('left')
    cbar.set_label('Number of Dashcam Images', fontsize=14, labelpad=10)
    cbar.ax.yaxis.set_label_position('left')
    cbar.ax.yaxis.set_tick_params(labelsize=12)

# Create histogram axis to the right of colorbar
hist_width = 0.2  # Width of histogram
if SHOW_COLORBAR:
    hist_x = cax.get_position().x1 + 0.005  # Start right at right edge of colorbar
    hist_y = cax.get_position().y0
else:
    hist_x = 0.2  # Same starting position as where colorbar would be
    hist_y = 0.65  # Same y-position as where colorbar would be
hist_ax = fig.add_axes([hist_x,
                       hist_y,
                       hist_width,
                       0.2])  # Same height as colorbar
                       cax.get_position().y0,
                       hist_width,
                       cax.get_position().height])

# Calculate and plot histogram bars
bar_height = 1 / (len(bins)-1)  # Make bars thinner than color segments
bin_counts = []
for i in range(len(bins)-1):
    bin_mask = (cts_nyc['nframes'] >= bins[i]) & (cts_nyc['nframes'] < bins[i+1])
    count = np.sum(bin_mask)
    bin_counts.append(count)
    color = cmap(norm(bins[i]))
    
    # Calculate the center position
    y_pos = (i + 0.5)/(len(bins)-1)  # This will align with colorbar segments
    
    hist_ax.barh(y_pos,  # Center in color segment
                count,
                height=bar_height,
                color=color,
                edgecolor='black',
                linewidth=0.5)

# Set the y-axis limits to match colorbar
hist_ax.set_ylim(0, 1)

# Colorbar properties moved up to where other colorbar code is

# Configure histogram axis and add ticks if colorbar is hidden
HIST_FRAME_ON=True
hist_ax.spines['top'].set_visible(HIST_FRAME_ON)
hist_ax.spines['right'].set_visible(HIST_FRAME_ON)
hist_ax.spines['bottom'].set_visible(HIST_FRAME_ON)
hist_ax.spines['left'].set_visible(HIST_FRAME_ON)

if not SHOW_COLORBAR:
    # Calculate tick positions at center of each color block
    tick_positions = [(i + 0.5)/(len(bins)-1) for i in range(len(bins)-1)]
    hist_ax.set_yticks(tick_positions)
    
    # Create range labels
    tick_labels = [format_range(bins[i], bins[i+1]) for i in range(len(bins)-1)]
    hist_ax.set_yticklabels(tick_labels, rotation=0)
    
    # Style the ticks
    hist_ax.yaxis.set_ticks_position('left')
    hist_ax.yaxis.set_tick_params(labelsize=12)
    hist_ax.set_ylabel('Number of Dashcam Images', fontsize=14, labelpad=10)
else:
    hist_ax.set_yticks([])

hist_ax.grid(False, axis='x', linestyle='--', alpha=0.3)
hist_ax.set_xlabel('Number of\nNeighborhoods', fontsize=14)

# Add more frequent x ticks
max_count = max(bin_counts)
hist_ax.set_xticks([0, 250, 500, 750, 1000, 1250, 1500])

# Only access colorbar properties if it exists
if SHOW_COLORBAR:
    cbar.ax.yaxis.set_tick_params(labelsize=12)

plt.savefig(f"{constants.PAPER_PATH}/figures/nta_nframes_hist.pdf", dpi=250, bbox_inches='tight', pad_inches=0.025)